In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName("DataFrame Intro").getOrCreate()

df = spark.read.option("delimiter","\t").\
            option("header","true").\
            option("inferSchema","true").\
            csv("../datasets/DepremVerileri.txt")
#df.printSchema()
#df.select("Olus tarihi","Enlem","Boylam","xM").show()

def bolgeAtama(enlem,boylam):
    bolge =""
    if enlem <= 39 :
        if boylam <= 31 :
            bolge = "Bolge 1"
        elif boylam <= 36 :
            bolge = "Bolge 2"
        elif boylam <= 41 :
            bolge = "Bolge 3"
        else :
            bolge = "Bolge 4"
    else:
        if boylam <= 31 :
            bolge = "Bolge 5"
        elif boylam <= 36 :
            bolge = "Bolge 6"
        elif boylam <= 41 :
            bolge = "Bolge 7"
        else :
            bolge = "Bolge 8"
    return bolge


udfBogeAtama = UserDefinedFunction(bolgeAtama,StringType())
df.createOrReplaceTempView("depremler")
df2 = spark.sql("select * from depremler")
df3 = df2.select("Olus tarihi","Enlem","Boylam","xM")\
                .sort("xM",ascending=False)

#df3 = df3.withColumn("Bolge",udfBogeAtama("Enlem","Boylam"))
#df3.select("Bolge").groupBy("Bolge").count().sort("count").show()

df3.withColumn("sum", df3["Enlem"]+df3["Boylam"]).show()

+-----------+-------+------+---+-----------------+
|Olus tarihi|  Enlem|Boylam| xM|              sum|
+-----------+-------+------+---+-----------------+
| 1939.12.26|   39.8| 39.51|7.9|            79.31|
| 1926.06.26|  36.54| 27.33|7.7|            63.87|
| 1930.05.06|  37.98| 44.48|7.6|            82.46|
| 1976.11.24|  39.05| 44.04|7.5|            83.09|
| 1999.08.17|  40.76| 29.97|7.4|70.72999999999999|
| 1912.08.09|   40.6|  27.2|7.3|             67.8|
| 1943.11.26|  41.05| 33.72|7.2|            74.77|
| 2011.10.23|38.7212|43.411|7.2|82.13220000000001|
| 1944.02.01|  41.41| 32.69|7.2|             74.1|
| 1999.11.12|  40.74| 31.21|7.2|            71.95|
| 1953.03.18|  39.99| 27.36|7.2|            67.35|
| 1948.02.09|  35.41|  27.2|7.2|            62.61|
| 1957.04.25|  36.42| 28.68|7.1|             65.1|
| 1916.01.24|  40.27| 36.83|7.1|             77.1|
| 1957.05.26|  40.67|  31.0|7.1|            71.67|
| 1942.12.20|  40.87| 36.47|7.0|            77.34|
| 1970.03.28|  39.21| 29.51|7.0